In [ ]:
# 1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import plotly.express as px

# Settings
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set(style="whitegrid")

# 2. Load Data
df = pd.read_csv('../data/sierraleone-bumbuna.csv')  # Adjust path as needed
df.head()

# 3. Summary Statistics & Missing Values
print("Summary Statistics:")
display(df.describe())

print("\nMissing Values:")
missing = df.isna().sum()
print(missing[missing > 0])
print("\nColumns with >5% Nulls:")
print((missing / len(df) * 100)[missing > 0])

# 4. Z-Score Outlier Detection and Basic Cleaning
cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
df_z = df[cols].apply(zscore)
outliers = (np.abs(df_z) > 3).any(axis=1)
print(f"Outliers detected: {outliers.sum()} rows")

df_clean = df[~outliers].copy()

# 5. Median Imputation
df_clean.fillna(df_clean.median(numeric_only=True), inplace=True)

# 6. Export Cleaned CSV (Do NOT commit to Git)
df_clean.to_csv('../data/sierraleone_clean.csv', index=False)

# 7. Time Series Plot
df_clean['Timestamp'] = pd.to_datetime(df_clean['Timestamp'])
df_clean.set_index('Timestamp')[['GHI', 'DNI', 'DHI', 'Tamb']].plot(figsize=(14, 6), title="Solar Irradiance & Temperature")
plt.ylabel('Irradiance / Temp')
plt.show()

# 8. Cleaning Impact Analysis (if Cleaning flag exists)
if 'Cleaning' in df_clean.columns:
    df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean().plot(kind='bar', title='ModA & ModB Before/After Cleaning')
    plt.ylabel('Sensor Reading')
    plt.show()

# 9. Correlation Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df_clean[cols + ['TModA', 'TModB']].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

# 10. Scatter Plots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
sns.scatterplot(data=df_clean, x='WS', y='GHI', ax=axs[0])
sns.scatterplot(data=df_clean, x='WSgust', y='GHI', ax=axs[1])
sns.scatterplot(data=df_clean, x='WD', y='GHI', ax=axs[2])
axs[0].set_title("WS vs GHI")
axs[1].set_title("WSgust vs GHI")
axs[2].set_title("WD vs GHI")
plt.tight_layout()
plt.show()

# 11. RH Correlation
sns.scatterplot(data=df_clean, x='RH', y='Tamb')
plt.title("Relative Humidity vs Temperature")
plt.show()

sns.scatterplot(data=df_clean, x='RH', y='GHI')
plt.title("Relative Humidity vs GHI")
plt.show()

# 12. Wind & Distribution Analysis
# Histograms
df_clean['GHI'].hist(bins=30, alpha=0.6, label='GHI')
df_clean['WS'].hist(bins=30, alpha=0.6, label='WS')
plt.legend()
plt.title("Distribution of GHI and WS")
plt.show()

# Wind Rose – Requires additional packages like `windrose`, skipping here.

# 13. Bubble Chart: GHI vs Tamb with RH as Bubble Size
px.scatter(df_clean, x='Tamb', y='GHI', size='RH', color='RH',
           title='GHI vs Tamb (Bubble Size = RH)', 
           labels={'Tamb': 'Temperature', 'GHI': 'Global Horizontal Irradiance'}).show()
